# Download Hourly Canadian Weather Data

This notebook can be used to browse and download hourly weather data from Environment Canada Weather Stations

In [59]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import datetime
# import os
import folium
import requests

%matplotlib notebook

token = "pk.eyJ1IjoianVuZXNwYWNlYm9vdHMiLCJhIjoiY2s4a285NTM1MDQwbDNocHozdXlkNzIyaSJ9.-_8gh9gG4VuprIPnmXBr3A" # your mapbox token
tileurl = 'https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}@2x.png?access_token=' + str(token)


# Define Query

## Search Areas

## Met Stations
* Filter by start year (optional)
* Calculate record lengths

In [60]:
## Search Areas (e.g. UBC Micromet flux towers)
Towers =  gpd.read_file('https://raw.githubusercontent.com/ubc-micromet/FieldSiteMaps/main/data/FluxTowers.json')
print(Towers)
Towers = Towers.set_index('Name')

## Met Stations
Start_Year = 2014
End_Year = datetime.datetime.now().year

Stations = pd.read_csv('station_list/Stations.csv',skiprows=0)

Dates = ['FIRST_DATE','LAST_DATE','HLY_FIRST_DATE','HLY_LAST_DATE','DLY_FIRST_DATE','DLY_LAST_DATE',
         'MLY_FIRST_DATE','MLY_LAST_DATE']

for date in Dates:
    Stations[date] = pd.to_datetime(Stations[date])
    
Stations['HLY_REC_LENGTH'] = Stations['HLY_FIRST_DATE']-Stations['HLY_LAST_DATE']

Stations = Stations.rename(columns={'y':'Latitude (Decimal Degrees)',
                                    'x':'Longitude (Decimal Degrees)'})

# Filter Stations
Stations = Stations.loc[((Stations['HLY_FIRST_DATE'].dt.year<=Start_Year)&
                        (Stations['HLY_LAST_DATE'].dt.year>=Start_Year)&
                        (Stations['ENG_PROV_NAME']=='BRITISH COLUMBIA'))].copy()

Stations = Stations.set_index(Stations['STATION_NAME'],drop = True)

gdfQuery = gpd.GeoSeries(index=Query['Site'].values,
        data=gpd.points_from_xy(Query['Longitude (Decimal Degrees)'],
                                Query['Latitude (Decimal Degrees)']
                               ),
                    crs='WGS1984'
                                  )

gdf = gpd.GeoDataFrame(Stations,
    geometry=gpd.points_from_xy(Stations['Longitude (Decimal Degrees)'],
                                Stations['Latitude (Decimal Degrees)']
                               ),
                    crs='WGS1984'
                                  )
# Now we can assign a CRS
WGS_1984='epsg:4326'
gdfQuery.set_crs = WGS_1984
gdf.crs = WGS_1984


   OBJECTID Name   Longitude   Latitude  \
0         4  BB1 -122.984947  49.129360   
1         5  BB2 -122.995163  49.119015   
2         6  DSM -122.895210  49.088196   
3         7  RBM -123.196411  49.131161   

                                              Access  \
0  First gate is a yellow gate on 80th Street nex...   
1  First gate is a yellow gate on 80th Street nex...   
2  Enter the gate through 112th Street, follow th...   
3  Get to the West Dyke Trail through Steveston H...   

                                             WebPlot  Start  \
0  https://ibis.geog.ubc.ca/~micromet/data/burnsb...   2014   
1  https://ibis.geog.ubc.ca/~micromet/data/burnsb...   2019   
2  https://ibis.geog.ubc.ca/~micromet/data/DeltaS...   2021   
3  https://ibis.geog.ubc.ca/~micromet/data/Richmo...   2022   

                                         Description  \
0  A harvested peatland that is undergoing active...   
1  A harvested peatland that is not undergoing ac...   
2             A deg

# Reproject, Buffer by Radius, and Search

In [65]:

## Set to BC Albers
gdf = gdf.to_crs('epsg:3153')
gdfQuery = gdfQuery.to_crs('epsg:3153')
TowersQuery = Towers.to_crs('epsg:3153')


# print(gdfQuery.items())

## Search a Radius of 20km
Dist = 1.5e4
gdfQuery = gdfQuery.buffer(Dist)
TowersQuery = TowersQuery.buffer(Dist)

Select = gdf.assign(**{(key): gdf.within(geom) for key, geom in TowersQuery.geometry.items()})

Selection = Select[Select[Towers.index.values].sum(axis=1)>0].copy()

Selection.T

STATION_NAME,DELTA BURNS BOG,VANCOUVER INTL A,VANCOUVER SEA ISLAND CCG,WHITE ROCK CAMPBELL SCIENTIFIC
Longitude (Decimal Degrees),-123.002246,-123.183889,-123.187236,-122.783889
Latitude (Decimal Degrees),49.125848,49.194722,49.1825,49.018056
STN_ID,49088,51442,51357,925
STATION_NAME,DELTA BURNS BOG,VANCOUVER INTL A,VANCOUVER SEA ISLAND CCG,WHITE ROCK CAMPBELL SCIENTIFIC
PROV_STATE_TERR_CODE,BC,BC,BC,BC
ENG_PROV_NAME,BRITISH COLUMBIA,BRITISH COLUMBIA,BRITISH COLUMBIA,BRITISH COLUMBIA
COUNTRY,CAN,CAN,CAN,CAN
LATITUDE,490733053,491141000,491057000,490105000
LONGITUDE,-1230008085,-1231102000,-1231114050,-1224702000
TIMEZONE,PST,PST,PST,PST


# Display Results

In [66]:
Map = folium.Map(
    location=[49.12917,-122.98486], zoom_start=10)#, tiles=tileurl, attr='Mapbox')


for i,row in Towers.iterrows():
    print(row['Latitude'],row['Longitude'])
    point=folium.CircleMarker(
            location=[row['Latitude'],row['Longitude']],
            radius=10,
            popup=i,
            fill_opacity = 1,
            fill=True,
            color='black',
            line_weight=.35,
            fill_color='green'
    ).add_to(Map)

for i,row in Selection.iterrows():
    point=folium.CircleMarker(
            location=[row['Latitude (Decimal Degrees)'],row['Longitude (Decimal Degrees)']],
            radius=5,
            popup=row['STATION_NAME']+'\n'+str(row['HLY_REC_LENGTH']) +' '+str(row['HLY_FIRST_DATE'])+' '+str(row['HLY_LAST_DATE'])+\
        ' '+str(row['CLIMATE_IDENTIFIER']),
            fill_opacity = 1,
            fill=True,
            color='black',
            line_weight=.35,
            fill_color='blue'
    ).add_to(Map)
Map

49.12936019897461 -122.98494720458984
49.119014739990234 -122.99516296386719
49.08819580078125 -122.89521026611328
49.131160736083984 -123.1964111328125


# Download Data

In [68]:
for i,row in Selection.iterrows():
    Root = 'https://api.weather.gc.ca/collections/climate-hourly/items?'

    Start_Year = 2014
    End_Year = 2022

    Range = 'datetime='+str(Start_Year)+'-01-01%2000:00:00/'+str(End_Year)+'-12-31%2000:00:00'

    Station = '&CLIMATE_IDENTIFIER='+str(row['CLIMATE_IDENTIFIER'])

    End = '&sortby=LOCAL_DATE&f=csv&limit=100000&startindex=0'

    Download = (Root+Range+Station+End)
    response = requests.get(Download)
    open("Data/"+str(i)+".csv", "wb").write(response.content)
    print('Downloaded: ',i)
    

Downloaded:  DELTA BURNS BOG
Downloaded:  VANCOUVER INTL A
Downloaded:  VANCOUVER SEA ISLAND CCG
Downloaded:  WHITE ROCK CAMPBELL SCIENTIFIC
